# SNAPy Introduction
24/09/2024

This will introduce the basic uses of snapy-toolbox library, on importing data, preparing it, creating class, and intial map plotting.

#### 1. Import library
To import SNAPy, call import 'SNAPy', which is differnt from the pypi name of snapy-toolbox.<br>
Other than that, also import geopandas for reading and accessing GIS data

In [5]:
import SNAPy as sna
import geopandas as gpd

#### 2. Import files
Import any GIS data that can be read with geopandas.read_file. GeoJson, gpkg, shp, etc. can be read.
The data needed for initializing the graphsim class at least contains two seperate layers of:
-  network/edges data, usually roadlines that can be extracted from OSM
-  entry data, from building/parcel centroid or entrance points.

Note that the data has to be converted to geographic projections, with units of feet or meters. Lat-Lon coordinates will be rejected on GraphSims initialization. Both layer projections also need to be the same.

In [6]:
MainDir = 'https://raw.githubusercontent.com/kevinsutjijadi/SNAPyDocs/main/Samples/SampleData/'
dfNetwork = gpd.read_file(MainDir+'CentralJakarta_Roadline.gpkg')
dfEntries = gpd.read_file(MainDir+'CentralJakarta_Entries.gpkg')

The sample data provided here is obtained from OSM query on 09/2024, and converted/reprojected to epsg:32748.<br>
To read sample of the data:

In [7]:
dfEntries

,wadmkc,wadmkk,Area_Sqm,Function,geometry
0,Tanah Abang,Jakarta Pusat,122,Residential,POINT (700722.633 9315497.361)
1,Tanah Abang,Jakarta Pusat,119,Residential,POINT (700735.086 9315545.773)
2,Tanah Abang,Jakarta Pusat,80,Residential,POINT (701091.752 9315597.168)
3,Tanah Abang,Jakarta Pusat,32,Residential,POINT (700638.353 9315353.774)
4,Tanah Abang,Jakarta Pusat,198,Commercial,POINT (700905.896 9315498.524)
...,...,...,...,...,...
40700,Senen,Jakarta Pusat,29,Residential,POINT (704184.612 9316807.924)
40701,Kemayoran,Jakarta Pusat,203,Residential,POINT (703727.878 9318385.482)
40702,Senen,Jakarta Pusat,79,Residential,POINT (703525.85 9316833.666)
40703,Kemayoran,Jakarta Pusat,197,Residential,POINT (703820.534 9317540.427)


#### 3. Initializing GraphSims
GraphSims is the main class for analysis functions. Before running any analysis, we first need to build the network, such as appending relations between edges and mapping entries to the nearest edges.

Note that because this is the first time running using the roadline data that is acquired from OSM, not all lines have been segmented correctly on intersections between lines. GraphSims initialization will detect if there are more than 30% line ends are deadends, it will prompt an input to initiate segmentation process. On jupyter environments, this will appear above/overhead with inputs needed:
- y/[n] to segment lines: type 'y' and enter to segment (if 'n' or enter, it will skip segmenting and continue using the network as is)
- Line extention, default 0.5: enter float of distance for extenting line ends for imprecise drawings (if enter directly will use the deafult 0.5)
- Double iteration y/[n]: type 'y' to double iterate intersections (only use when there are self-connected lines in the model, note this will lengthen the segmenting process duration).

In [8]:
nwSim = sna.GraphSims(dfNetwork, dfEntries) 

DataFrame contains MultiLineString, exploding to LineString
Warning, more than 30% of endpoints are dead ends, segment intersections first?
segmenting intersections
	Splitting lines
	6123/6123 | 100.0%
	NetworkSegmentIntersection splits from 6135 lines to 10278 lines
	Running double pass
	Splitting lines
	10266/10266 | 100.0%
	NetworkSegmentIntersection splits from 10278 lines to 10285 lines
Access segmented network at GraphSims.NetworkDf and GraphSims.ixDf, recommended to save both dataframes, future GraphSims runs use segmented datasets
GraphSim Class ----------
Projection EPSG:32748
EntriesDf EntID not detected, adding from index
NetworkDf EdgeID not detected, adding from index
Graph Built with 7,823 Nodes, 10,075 Edges
Graph mapped 40,705 Entries
Detected 2 Entries unable to map to network. Coordinate error or further than EntDist parameter
	Entry ids are :
	[3841, 27441]
Graph initialization finished in 1.9419 s


If GraphSims goes through/uses edges segmentation, it is recommended to save the segmented network, and further analysis/process should use that data instead.

In [9]:
nwSim.NetworkDf.to_file('SampleData\CentralJakarta_Roadline_Segmented.gpkg', layer='segmented', driver='GPKG')

On cell [4], we have created a GraphSims class.<br>
try accessing the entries data and notice that there are new fields appended on the geodataframe, where there are xLn_ID, that refers to the edge it is connected to, xPt_X and xPt_Y are the coordinates of the point it is connected to the edge. You can save the dataframe and open it in GIS files to show.

In [10]:
nwSim.EntriesDf

,wadmkc,wadmkk,Area_Sqm,Function,geometry,fid,xLn_ID,xPt_X,xPt_Y
0,Tanah Abang,Jakarta Pusat,122,Residential,POINT (700722.633 9315497.361),0,3106,700719.788047,9.315517e+06
1,Tanah Abang,Jakarta Pusat,119,Residential,POINT (700735.086 9315545.773),1,3102,700734.032316,9.315565e+06
2,Tanah Abang,Jakarta Pusat,80,Residential,POINT (701091.752 9315597.168),2,7309,701086.801761,9.315597e+06
3,Tanah Abang,Jakarta Pusat,32,Residential,POINT (700638.353 9315353.774),3,3147,700637.311531,9.315354e+06
4,Tanah Abang,Jakarta Pusat,198,Commercial,POINT (700905.896 9315498.524),4,1290,700907.551151,9.315477e+06
...,...,...,...,...,...,...,...,...,...
40700,Senen,Jakarta Pusat,29,Residential,POINT (704184.612 9316807.924),40700,3401,704165.936042,9.316792e+06
40701,Kemayoran,Jakarta Pusat,203,Residential,POINT (703727.878 9318385.482),40701,4957,703693.648452,9.318373e+06
40702,Senen,Jakarta Pusat,79,Residential,POINT (703525.85 9316833.666),40702,7296,703556.439702,9.316786e+06
40703,Kemayoran,Jakarta Pusat,197,Residential,POINT (703820.534 9317540.427),40703,1654,703819.769234,9.317548e+06


#### 4. Map Preview
To visually confirm that the graph has been built correctly, we can use nwSim.Map_Show() to give us preview of the network. add 'junction' argument to show intersection nodes.

In [ ]:
nwSim.Map_Show('junction')
# not run on this jupyter, due to large size causing the notebook have larger than 100mb size for github upload. Run locally.

#### 5. Sample Analysis
After building GraphSims, we can directly use analysis tools. For this example, we use betweenees patronage.
Before using function, we determine:
- entries and destination ids
- weight to use
- calculation parameters

In [11]:
# getting unique values from 'function' field
nwSim.EntriesDf.Function.unique()

array(['Residential', 'Commercial', 'Amenity', 'Other', 'Special',
       'Religious', None], dtype=object)

In [12]:
# if we want origin of residential to commercial and amenity, we can get index list by using pandas filter techniques:
entrydf = nwSim.EntriesDf

idOrigin = entrydf[entrydf['Function']=='Residential'].fid
idDestination = entrydf[entrydf['Function'].isin(('Commercial', 'Religious'))].fid
print(f'Origin : {len(idOrigin):,} points| Destination : {len(idDestination):,} points')

Origin : 28,716 points| Destination : 10,157 points


In [13]:
baseSet = {
    'DetourR' : 1.1, # DetourR for detour ratio
    'SearchDist' : 400.0, # SearchDist for search distance
    'OriWgt': 'Area_Sqm', # OriWgt and DestWgt are for origin and destination weights
    'DestWgt': 'Area_Sqm',
    'RsltAttr': 'Btwn_R-C', # RsltAttr for the name of the result field
}
nwSim.BetweenessPatronage(idOrigin, idDestination, **baseSet)

BetweenessPatronage ---------- 
As Btwn_R-C from Area_Sqm to Area_Sqm
Collected 28704 Origin and 10157 Destinations Point[s]
Processing with multithreading & Plural mapping, with chunksize 624
Multiprocessing gph_Base_BetweenessPatronage_Plural_multi, on 46 task chunks
Multiprocessing finished in 3.709 seconds


,level_0,index,osm_id,osm_type,highway,geometry,fid,Btwn_R-C
0,0,0,4705040,way,primary,"LINESTRING (703800.365 9316751.297, 703801.516...",0,0.000000
1,1,1,4705040,way,primary,"LINESTRING (703772.644 9316922.824, 703766.473...",1,0.000000
2,2,2,4705040,way,primary,"LINESTRING (703766.473 9316956.468, 703758.763...",2,0.000000
3,3,3,4705040,way,primary,"LINESTRING (703756.018 9317015.631, 703749.136...",3,0.000000
4,4,4,4705043,way,tertiary,"LINESTRING (703860.81 9317892.838, 703793.691 ...",4,9545.091797
...,...,...,...,...,...,...,...,...
10070,10070,10280,1305006324,way,service,"LINESTRING (704043.075 9315384.838, 704066.424...",10070,4866.884277
10071,10071,10281,1305006325,way,service,"LINESTRING (704015.265 9315370.745, 704022.675...",10071,2896.908936
10072,10072,10282,1306603304,way,living_street,"LINESTRING (700565.364 9314607.102, 700565.089...",10072,1648.332153
10073,10073,10283,1307416756,way,service,"LINESTRING (702571.455 9316717.414, 702580.983...",10073,0.000000


#### 6. Saving Results
All analysis functions returns the dataframe that contains the new result attribute.
This can be directly accessed from the nwSim.NetworkDf, nwSim.EntryDf, and nwSim.NodeDf.
These are all geodataframe that can be saved using to_file

In [14]:
nwSim.NetworkDf.to_file("Intro_Result.gpkg", layer="Edges", driver='GPKG')
nwSim.EntriesDf.to_file("Intro_Result.gpkg", layer="Entries", driver='GPKG')
nwSim.NodeDf.to_file("Intro_Result.gpkg", layer="Nodes", driver='GPKG')